In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
import torch.nn as nn

from modules.unet import UNet
from modules.percep_nets import ResidualsNetUp, ResidualsNetDown
from transfers import Transfer, pretrained_transfers
from logger import Logger, VisdomLogger
from datasets import load_train_val, load_test, load_ood, load_all, TaskDataset
from task_configs import tasks, RealityTask
from model_configs import model_types
from models import DataParallelModel
from utils import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_CUDA = torch.cuda.is_available()

In [3]:
JOB

'YKIVVA_multitask:x->n&r_incremental'

In [4]:
logger = VisdomLogger("Jupyter_vis", env="YKIVVA_jupyter_visualize_", port=PORT, server=SERVER)

Setting up a new session...


No deletion
In (git) scaling-reset
Logging to environment YKIVVA_jupyter_visualize_


# DATASET

In [5]:
tasks_set = [tasks.rgb, tasks.normal, tasks.depth_zbuffer]

In [6]:
# test_set = load_test(tasks_set, buildings=['almena', 'albertville', 'espanola'])
# test = RealityTask.from_static(
#     "test", test_set, tasks_set
# )

In [7]:
buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

    rgb file len: 8785
    Intersection files len:  8785


In [8]:
#test_iter = iter(test_loader)
#rgb_data, normal_data, depth_data = next(test_iter)

# DOWNLOAD MODELS

In [7]:
project_dir = "/scratch/kivva/projects/consistency_LS"
model_class = "unet_based"
pipeline = "multitask"
res_dir = os.path.join(project_dir, model_class, pipeline,)
model_type = model_types[model_class]
final_dir = os.path.join(res_dir, "results_YKIVVA_unet3_multitask:x->n&r_v0_/models")

In [8]:
rgb_down = DataParallelModel(model_type["rgb"]["down"][0]())
normal_down = DataParallelModel(model_type["normal"]["down"][0]())
depth_down = DataParallelModel(model_type["depth_zbuffer"]["down"][0]())
normal_up = DataParallelModel(model_type["normal"]["up"][0]())
depth_up = DataParallelModel(model_type["depth_zbuffer"]["up"][0]())

In [9]:
rgb_down.load_weights(final_dir+"/rgb2LS.pth")
#normal_down.load_weights(RESULTS_DIR_MODELS+"/normal2LS.pth")
#depth_down.load_weights(RESULTS_DIR_MODELS+"/depth_zbuffer2LS.pth")
normal_up.load_weights(final_dir+"/LS2normal.pth")
depth_up.load_weights(final_dir+"/LS2depth_zbuffer.pth")

# VISUALIZE RESULTS

In [68]:
rgb_data = test.task_data[tasks.rgb].to(DEVICE)
normal_data = test.task_data[tasks.normal].to(DEVICE)
depth_data = test.task_data[tasks.depth_zbuffer].to(DEVICE)

In [69]:
with torch.no_grad():
    rgb_out = rgb_down(rgb_data)
    normal_out = normal_up(rgb_out)
    depth_out = depth_up(rgb_out)
    
    shape = list(rgb_data.shape)
    shape[1] = 3
    normal_out = normal_out.clamp(min=0, max=1).expand(*shape)
    depth_out = depth_out.clamp(min=0, max=1).expand(*shape)
    depth_data = depth_data.clamp(min=0, max=1).expand(*shape)

In [70]:
images = [depth_data, depth_out, normal_data, normal_out]

logger.images_grouped(images, f"results:r, r(x), n, n(x)", resize=256)

# PREDICTION RESULTS

In [12]:
src_task = tasks.rgb
target_task = tasks.depth_zbuffer
tasks_set = [src_task, target_task]

buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

    rgb file len: 8785
    Intersection files len:  8785


In [13]:
test_iter = iter(test_loader)

accuracy = 0
for in_data, out_data in test_iter:
    in_data = in_data.to(DEVICE)
    out_data = out_data.to(DEVICE)
    if src_task==tasks.rgb:
        model_down = rgb_down.to(DEVICE)
    if src_task==tasks.normal:
        model_down = normal_down.to(DEVICE)
    if src_task==tasks.depth_zbuffer:
        model_down = depth_down.to(DEVICE)
    
    if target_task==tasks.normal:
        model_up = normal_up.to(DEVICE)
    if target_task==tasks.depth_zbuffer:
        model_up = depth_up.to(DEVICE)
    
    with torch.no_grad():
        out = model_up(model_down(in_data))
    accuracy_tmp, _ = target_task.norm(out, out_data, compute_mse=False, batch_mean=False)
    accuracy += accuracy_tmp.sum()

print(accuracy / 8785. * 100.)

tensor(5.5815, device='cuda:0')
